In [2]:
import numpy as np

In [14]:
def ex(col_idx:int, kline_data_lv2):
    column_values = kline_data_lv2[::-1, col_idx]
    
    diffs = np.diff(column_values)
    is_decreasing = diffs < 0
    long_point = np.argmax(~is_decreasing) + 1 if np.any(~is_decreasing) else len(column_values)
    
    is_decreasing = diffs > 0
    short_point = np.argmax(~is_decreasing) + 1 if np.any(~is_decreasing) else len(column_values)
    
    if long_point > short_point:
        return (True, long_point)
    else:
        return (False, short_point)

In [16]:
import numpy as np

# 예제 데이터
kline_data_lv2 = np.array([
    [10, 27, 30],
    [9, 24, 29],
    [8, 25, 28],
    [7, 26, 27],
    [6, 27, 26]
])

result = ex(0, kline_data_lv2)
print(result)  # (True, 4) 또는 (False, 2)

(False, 5)


In [17]:
for i in range(1, 5, 1):
    print(i)

1
2
3
4


In [1]:
a = {}
a['b'] = 1
a['c'] = 2

In [2]:
a

{'b': 1, 'c': 2}